In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
damir = pd.read_sql("select * from apl.damir_age_stats where psp_spe_snds > 0 and psp_spe_snds <= 42", config.connection_string)
damir

,psp_spe_snds,ann,age_ben_snds,nb,total_nb,ratio
0,1,20,0,20006799,170874330,0.117085
1,1,20,20,17127019,170874330,0.100232
2,1,20,30,17833626,170874330,0.104367
3,1,20,40,18584273,170874330,0.108760
4,1,20,50,22067147,170874330,0.129143
...,...,...,...,...,...,...
1075,42,24,50,560449,3753047,0.149332
1076,42,24,60,653318,3753047,0.174077
1077,42,24,70,631355,3753047,0.168225
1078,42,24,80,352557,3753047,0.093939


In [3]:
sql =f"""
    select sum(p20_pop) pop, sum(p20_pop0002)/sum(p20_pop) p0002, sum(p20_pop0305)/sum(p20_pop) p0305, sum(p20_pop0610)/sum(p20_pop) p0610, sum(p20_pop1117)/sum(p20_pop) p1117, sum(p20_pop1824)/sum(p20_pop) p1824, sum(p20_pop2539)/sum(p20_pop) p2539, sum(p20_pop4054)/sum(p20_pop) p4054, sum(p20_pop5564)/sum(p20_pop) p5564, sum(p20_pop6579)/sum(p20_pop) p6579, sum(p20_pop80p)/sum(p20_pop) p80p
    from iris.pop_iris
"""
pop_iris = pd.read_sql(sql, config.connection_string)
pop_iris

,pop,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,67154381,0.03206,0.034704,0.061382,0.086891,0.080617,0.180436,0.195758,0.126325,0.140975,0.06097


In [4]:
props=pop_iris.iloc[-1].to_dict()
props

{'pop': 67154381.0,
 'p0002': 0.03206025754298889,
 'p0305': 0.0347039421002183,
 'p0610': 0.061381639419772184,
 'p1117': 0.08689129306396258,
 'p1824': 0.08061663169823574,
 'p2539': 0.180436165437963,
 'p4054': 0.19575836757396364,
 'p5564': 0.12632508666858236,
 'p6579': 0.14097473402368194,
 'p80p': 0.06096971916098817}

In [5]:
def compute_overrepresentation(damir_dataset, year, psp_spe_snds):
    bb = 1 if psp_spe_snds==1 or psp_spe_snds==12 else 0
    damir = damir_dataset[damir_dataset["ann"]==year]
    damir = damir[damir["psp_spe_snds"]==psp_spe_snds]
    overs={"year":year, "psp_spe_snds":psp_spe_snds}
    overs["p0002"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(3+bb*3)/20/(props["p0002"])).iloc[0]
    overs["p0305"]=(damir[damir["age_ben_snds"]==0]["ratio"]*3/20/(props["p0305"])).iloc[0]
    overs["p0610"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(5-bb)/20/(props["p0610"])).iloc[0]
    overs["p1117"]=(damir[damir["age_ben_snds"]==0]["ratio"]*(7-bb)/20/(props["p1117"])).iloc[0]
    overs["p1824"]=((damir[damir["age_ben_snds"]==0]["ratio"]*(2-bb)/20+damir[damir["age_ben_snds"]==20]["ratio"].iloc[0]*5/10)/(props["p1824"])).iloc[0]
    overs["p2539"]=((damir[damir["age_ben_snds"]==20]["ratio"]*6/10+damir[damir["age_ben_snds"]==30]["ratio"].iloc[0]*10/10)/(props["p2539"])).iloc[0]
    overs["p4054"]=((damir[damir["age_ben_snds"]==40]["ratio"]*10/10+damir[damir["age_ben_snds"]==50]["ratio"].iloc[0]*5/10)/(props["p4054"])).iloc[0]
    overs["p5564"]=((damir[damir["age_ben_snds"]==50]["ratio"]*5/10+damir[damir["age_ben_snds"]==60]["ratio"].iloc[0]*5/10)/(props["p5564"])).iloc[0]
    overs["p6579"]=((damir[damir["age_ben_snds"]==60]["ratio"]*5/10+damir[damir["age_ben_snds"]==70]["ratio"].iloc[0]*10/10)/(props["p6579"])).iloc[0]
    overs["p80p"]=((damir[damir["age_ben_snds"]==80]["ratio"]*10/10+damir[damir["age_ben_snds"]==99]["ratio"].iloc[0]*10/10)/(props["p80p"])).iloc[0]
    if psp_spe_snds == 12:
        overs["p0002"]+=overs["p1117"]
        overs["p0002"]+=overs["p1824"]
        overs["p0002"]+=overs["p2539"]
        overs["p0002"]+=overs["p4054"]
        overs["p0002"]+=overs["p5564"]
        overs["p0002"]+=overs["p6579"]
        overs["p0002"]+=overs["p80p"]
        overs["p1117"]=overs["p1824"]=overs["p2539"]=overs["p4054"]=overs["p5564"]=overs["p6579"]=overs["p80p"]=0
    return overs
    
overs = compute_overrepresentation(damir, 20, 1)
overs

{'year': 20,
 'psp_spe_snds': 1,
 'p0002': 1.0956075899997004,
 'p0305': 0.5060730766552142,
 'p0610': 0.38149802702651187,
 'p1117': 0.4042460442565535,
 'p1824': 0.6942745692808933,
 'p2539': 0.9117125219900553,
 'p4054': 0.8854342527005641,
 'p5564': 1.0902346989302283,
 'p6579': 1.5322086367093906,
 'p80p': 2.480882153711509}

In [6]:
def df_append_dict(df, dico):
    dico_df = pd.DataFrame([dico])
    return pd.concat([df, dico_df], ignore_index=True)

df = pd.DataFrame(columns=overs.keys())
df = df_append_dict(df, overs)
df

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1,1.095608,0.506073,0.381498,0.404246,0.694275,0.911713,0.885434,1.090235,1.532209,2.480882


In [7]:
years=damir["ann"].unique()
print(years)
psp_spe_sndss=damir["psp_spe_snds"].unique()
psp_spe_sndss

[20 21 22 23 24]


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 18, 31, 32,
       33, 34, 35, 36, 37, 38, 42], dtype=int64)

In [8]:
df = pd.DataFrame(columns=overs.keys())
for year in years:
    for i in psp_spe_sndss:
        overs = compute_overrepresentation(damir, year, i)
        df = df_append_dict(df, overs)
df

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,20,1,1.095608,0.506073,0.381498,0.404246,0.694275,0.911713,0.885434,1.090235,1.532209,2.480882
1,20,2,0.469867,0.434073,0.409027,0.404522,0.709924,0.944055,0.972548,1.212435,1.687649,1.804486
2,20,3,0.182540,0.168634,0.158904,0.157154,0.322420,0.482512,0.836701,1.367779,2.310546,3.181000
3,20,4,0.378390,0.349565,0.329395,0.325767,0.650505,0.850499,0.993940,1.305427,1.793680,1.932714
4,20,5,0.668425,0.617505,0.581875,0.575466,0.986590,1.059708,0.871547,1.027651,1.450523,1.784618
...,...,...,...,...,...,...,...,...,...,...,...,...
115,24,35,0.176851,0.163379,0.153952,0.152256,0.362095,0.599625,0.882348,1.283693,2.246276,2.987061
116,24,36,3.150461,2.910464,2.742530,2.712324,1.330876,0.617506,0.401609,0.332767,0.344178,0.230879
117,24,37,0.220805,0.203984,0.192214,0.190097,0.785038,1.174727,1.127259,1.264580,1.568684,1.515923
118,24,38,0.441158,0.407552,0.384036,0.379806,0.700952,1.076921,0.940145,1.132410,1.627653,1.921922


In [9]:
df.to_sql("overrepresentation", config.connection_string, schema="apl", if_exists="replace", index=False)

120